In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
NB_USERS = 5000
FEAT_USER = 3
NB_ITEMS = 100
FEAT_ITEM = 6

NB_EPOCHS = 500
LAMBDA_REG = 1e-5
# LAMBDA_REG = 0
# learning_rate = 0.001

In [3]:
# Load data
users = pd.read_csv('data/sushi/sushi3.udata', sep='\t', names=('uid', 'gender', 'age', 'time', 'old_prefecture', 'old_region', 'old_eastwest', 'prefecture', 'region', 'eastwest', 'same'))
items = pd.read_csv('data/sushi/sushi3.idata', sep='\t', names=('iid', 'name', 'style', 'major', 'minor', 'heaviness', 'frequency', 'price', 'popularity'))
R = pd.read_csv('data/sushi/sushi3b.5000.10.score', sep=' ', header=None)
triplets = []
for i, line in enumerate(np.array(R)):
    for j, v in enumerate(line):
        if v != -1:
            triplets.append((i, j, v))
df_ratings = pd.DataFrame(triplets, columns=('user', 'item', 'rating'))
train, test = train_test_split(df_ratings, test_size=0.2, shuffle=True)

In [4]:
# TF
#A is not used (only the "criterion on the items" => only B)
#A = tf.constant(np.array(users[['age', 'gender', 'region']]).astype(np.float32)) 
B = tf.constant(np.array(items[['style', 'major','heaviness', 'frequency', 'price', 'popularity']]).astype(np.float32))

weights = tf.get_variable('W', shape=[NB_USERS, FEAT_ITEM], dtype=np.float32, initializer=tf.truncated_normal_initializer(stddev=1))

alpha = tf.get_variable("alpha", shape=[FEAT_ITEM],
                            initializer=tf.truncated_normal_initializer(stddev=1))
beta = tf.get_variable("beta", shape=[FEAT_ITEM],
                            initializer=tf.truncated_normal_initializer(stddev=1))

user_batch = tf.placeholder(tf.int32, shape=[None])
item_batch = tf.placeholder(tf.int32, shape=[None])
rate_batch = tf.placeholder(tf.float32, shape=[None])

weight_users = tf.nn.embedding_lookup(weights, user_batch)

#beta_crit = tf.nn.embedding_lookup(item_bias, item_batch)
#alpha_crit = tf.nn.embedding_lookup(user_bias, user_batch)

feat_items = tf.nn.embedding_lookup(B, item_batch)
#feat_users = tf.nn.embedding_lookup(A, user_batch)

pred =tf.reduce_sum(tf.multiply(tf.nn.softmax(tf.multiply(feat_items, alpha)+beta),weight_users),1)

cost_l2 = tf.losses.mean_squared_error(rate_batch, pred)

regularizer=tf.nn.l2_loss(weight_users)

#l2_user = tf.nn.l2_loss(weight_users)
#l2_item = tf.nn.l2_loss(weight_items)
#l2_bias_user = tf.nn.l2_loss(bias_users)
#l2_bias_item = tf.nn.l2_loss(bias_items)
#regularizer = tf.add(l2_user, l2_item)
#regularizer = tf.add(regularizer, l2_bias_user)
#regularizer = tf.add(regularizer, l2_bias_item)
# regularizer = tf.nn.l2_loss(M)
penalty = tf.constant(LAMBDA_REG, dtype=tf.float32, shape=[])
cost = tf.add(cost_l2, tf.multiply(regularizer, penalty))

In [5]:
global_step = tf.train.get_global_step()
train_op = tf.train.AdamOptimizer(0.1).minimize(cost, global_step=global_step)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(NB_EPOCHS):
        _, train_pred, train_mse, reg, pen, train_cost = sess.run([train_op, pred, cost_l2, regularizer, penalty, cost], feed_dict={
            user_batch: train['user'],
            item_batch: train['item'],
            rate_batch: train['rating']
        })
        test_pred, test_mse = sess.run([pred, cost_l2], feed_dict={
            user_batch: test['user'],
            item_batch: test['item'],
            rate_batch: test['rating']
        })
        print('train rmse', train_mse ** 0.5, 'test rmse', test_mse ** 0.5)
        # print('reg', reg, 'full cost', train_cost)
        
    #print(weights.eval(session=sess))
    W_mat=weights.eval(session=sess)
    alpha_vec= alpha.eval(session=sess)
    beta_vec= beta.eval(session=sess)
    

train rmse 3.0930782320900163 test rmse 3.0293337083713316
train rmse 2.9967815301060186 test rmse 2.936614207370401
train rmse 2.9041518974932043 test rmse 2.84941626476841
train rmse 2.8167533078684217 test rmse 2.766557008416174
train rmse 2.7334436192786913 test rmse 2.6862265652663453
train rmse 2.6525333207578585 test rmse 2.606837792287635
train rmse 2.572498817851367 test rmse 2.5275450542294333
train rmse 2.4925012660047248 test rmse 2.448224561009294
train rmse 2.412397977420982 test rmse 2.369214389341402
train rmse 2.3324792070678377 test rmse 2.291079521101194
train rmse 2.2532605811606765 test rmse 2.2144390725500616
train rmse 2.175301462963915 test rmse 2.1398506713175873
train rmse 2.099120809799498 test rmse 2.0678005886556323
train rmse 2.0251702507882476 test rmse 1.998719997884608
train rmse 1.9538576506710241 test rmse 1.9330099438265
train rmse 1.885559041529495 test rmse 1.8710453927116586
train rmse 1.8206312039731543 test rmse 1.8131408216278582
train rmse 1.7

train rmse 1.188483784386439 test rmse 1.3202672645900602
train rmse 1.188484887726336 test rmse 1.320265413607146
train rmse 1.1884849378781253 test rmse 1.3202685286744615
train rmse 1.1884860412169516 test rmse 1.3202658650666328
train rmse 1.1884860412169516 test rmse 1.3202703796530084
train rmse 1.1884864925825398 test rmse 1.3202666776933196
train rmse 1.188486743341126 test rmse 1.3202708762565674
train rmse 1.188487395313202 test rmse 1.3202692058620358
train rmse 1.188487746374941 test rmse 1.32026943159115
train rmse 1.1884883481948239 test rmse 1.3202714180056914
train rmse 1.1884885989530183 test rmse 1.3202680320700197
train rmse 1.1884884986497468 test rmse 1.3202700184866665
train rmse 1.18848814758823 test rmse 1.3202686641120043
train rmse 1.188487846678276 test rmse 1.3202656393369088
train rmse 1.1884882980431786 test rmse 1.320267806340666
train rmse 1.188487746374941 test rmse 1.3202639237897449
train rmse 1.1884874956165665 test rmse 1.320263652913673
train rmse 

train rmse 1.1870803593475232 test rmse 1.3184212225076233
train rmse 1.187054701183431 test rmse 1.3183890332312143
train rmse 1.1870291428913207 test rmse 1.3183553964060337
train rmse 1.1870021780455227 test rmse 1.318321306597367
train rmse 1.1869761164671924 test rmse 1.318287984539869
train rmse 1.1869491504167518 test rmse 1.318252491327019
train rmse 1.1869226357139566 test rmse 1.3182182179929138
train rmse 1.186895919543127 test rmse 1.3181837628986997
train rmse 1.1868681481491545 test rmse 1.3181479051317275
train rmse 1.1868400245563238 test rmse 1.3181126794650757
train rmse 1.1868116491845953 test rmse 1.3180768649852244
train rmse 1.1867837753713009 test rmse 1.318041004310057
train rmse 1.1867554488795287 test rmse 1.3180049165422723
train rmse 1.1867272221637641 test rmse 1.31796760672094
train rmse 1.1866975381850253 test rmse 1.317931833524666
train rmse 1.1866691091776653 test rmse 1.3178945668615143
train rmse 1.1866397251239007 test rmse 1.317857389601427
train r

train rmse 1.1846064902470925 test rmse 1.3152610710387231
train rmse 1.1846034209679086 test rmse 1.3152550890848924
train rmse 1.184600754210686 test rmse 1.3152519621436531
train rmse 1.184597785549358 test rmse 1.3152471131041297
train rmse 1.1845954206782303 test rmse 1.31524235468348
train rmse 1.1845931564354928 test rmse 1.3152389558010469
train rmse 1.184590087121761 test rmse 1.315233744164257
train rmse 1.1845871184337005 test rmse 1.315230662491048
train rmse 1.1845852567102733 test rmse 1.3152264025191338
train rmse 1.184582841497143 test rmse 1.315222459766876
train rmse 1.1845803256448961 test rmse 1.3152189248754926
train rmse 1.1845780613733055 test rmse 1.3152147555042135
train rmse 1.184575948049249 test rmse 1.3152114018698977
train rmse 1.1845731805953787 test rmse 1.3152080482270305
train rmse 1.184571067262615 test rmse 1.3152043773378725
train rmse 1.1845691048788156 test rmse 1.3152015221947773
train rmse 1.184566538679711 test rmse 1.3151972168085484
train rms

In [6]:
print(W_mat)
print(alpha_vec)
print(beta_vec)

[[-4.8738337e-05  1.9294201e-05  1.3087435e-04  1.2566648e+00
   7.3985589e-06  2.0610560e-02]
 [ 3.2607506e-06 -3.3195956e-06 -2.0274478e-05  5.0081193e-01
  -1.3728210e-09 -1.5982183e-03]
 [-2.3860825e-05 -2.1961761e-05 -3.4835134e-04  2.2185493e+00
  -3.1537449e-05 -5.7667527e-02]
 ...
 [-1.6882614e-05  2.3747984e-06  2.5604702e-06  2.0864081e+00
  -3.3230835e-06 -1.0508324e-03]
 [ 1.0521260e-06 -1.4005027e-06 -1.4277983e-04  2.6976762e+00
  -3.9130026e-07 -1.1859034e-02]
 [-1.7295177e-06 -5.9381155e-06 -5.2427469e-05  2.5884979e+00
  -2.7851449e-06 -4.6925615e-03]]
[-3.222721   -2.2427385  -0.4189402   6.2142725  -1.7571095  -0.91200113]
[-0.95939165 -2.9488835   0.03732969  2.477148   -1.4513202   3.9942234 ]


In [10]:
#import matplotlib.pyplot as plt
#plt.imshow(W_mat)

# add some prints/visualisation
# check encoding B
# positivity on W 

In [ ]:
# TODO => comparer avec SVD++ / baseline constante (predire la moyenne des ratins)
# pourquoi le modele bilineaire n'était pas convaicant ?